#Подготовка данных и их разметка

In [51]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack

In [52]:
data = pd.read_parquet("https://huggingface.co/datasets/d0rj/povarenok_recipes_detail/resolve/main/data/train-00001-of-00002-8ce75ff1e8c43597.parquet")
data.head(3)
#Импортируем датасет

,title,page_url,main_image_url,description,creation_datetime,views,kroshki,ingredients,cooking_time,portions_count,cooking_steps,cooking_steps_images_urls,cooking_additional_images_urls,purposes,tags,tastes,nae_value,user_photos_urls,recipe_variants_urls
0,Мини-кутабы,https://www.povarenok.ru/recipes/show/92442/,https://www.povarenok.ru/data/cache/2014jul/24...,Весной и в начале лета практически все с больш...,2014-07-24T09:19:00,16290,"[Выпечка, Изделия из теста, Другие изделия]","[{'count': None, 'name': 'Мука пшеничная'}, {'...",30 минут,4,"[Смешиваем просеянную муку и воду, солим. Заме...",[https://www.povarenok.ru/data/cache/2014jul/2...,[/data/cache/2014jul/24/11/807097_10031nothumb...,"[Для детей, На завтрак, Конкурсные рецепты, Ко...",[завтрак],"[свежий, сочный]","{'100 г блюда': {'carb': 21.4, 'fats': 10.9, '...",[https://www.povarenok.ru/data/cache/2016apr/0...,[https://www.povarenok.ru/recipes/show/105939/...
1,Куриные бедрышки с овощами и прованскими травами,https://www.povarenok.ru/recipes/show/92444/,https://www.povarenok.ru/data/cache/2014jul/24...,Захотелось рецепт оформить роликом. Делала нес...,2014-07-24T11:21:00,2827,[Горячие блюда],"[{'count': '6 шт', 'name': 'Бедро куриное'}, {...",35 минут,3,[],[],[],"[На обед, На второе, На праздничный стол, 23 ф...","[ужин, обед, второе, курица, картофель]","[куриный, пряный, ароматный, сочный]","{'100 г блюда': {'carb': 6.9, 'fats': 6.0, 'kc...",[],[https://www.povarenok.ru/recipes/show/107223/...
2,Хлеб на яблочном соке,https://www.povarenok.ru/recipes/show/92443/,https://www.povarenok.ru/data/cache/2014jul/21...,"Вы не пробовали хлебушек на соке? Мягкий, пуши...",2014-07-24T09:32:00,5697,"[Выпечка, Изделия из теста, Домашний хлеб]","[{'count': '160 мл', 'name': 'Сок свежевыжатый...",110 минут,None,[Разводим дрожжи в теплой водичке и даем им по...,[https://www.povarenok.ru/data/cache/2014jul/2...,[/data/cache/2014jul/24/41/807009_54639nothumb...,"[Для детей, Конкурсные рецепты, Конкурс ""Больш...","[хлеб, дрожжи]","[нежный, ароматный]","{'100 г блюда': {'carb': 44.1, 'fats': 9.9, 'k...",[],"[https://www.povarenok.ru/recipes/show/73982/,..."


In [53]:
!pip install nltk
import nltk

In [54]:
dataframe = data[['title', 'description', 'ingredients','cooking_steps']][:10000].copy()
dataframe
#Обрезали датасет на 10.000 объектов. Дело в том, что далее мы столкнемся в очень большим временем обучения модели.

,title,description,ingredients,cooking_steps
0,Мини-кутабы,Весной и в начале лета практически все с больш...,"[{'count': None, 'name': 'Мука пшеничная'}, {'...","[Смешиваем просеянную муку и воду, солим. Заме..."
1,Куриные бедрышки с овощами и прованскими травами,Захотелось рецепт оформить роликом. Делала нес...,"[{'count': '6 шт', 'name': 'Бедро куриное'}, {...",[]
2,Хлеб на яблочном соке,"Вы не пробовали хлебушек на соке? Мягкий, пуши...","[{'count': '160 мл', 'name': 'Сок свежевыжатый...",[Разводим дрожжи в теплой водичке и даем им по...
3,"Коктейль ""Застенчивая дыня""",Этот коктейль вовсе не для детей! Вкусно и хме...,"[{'count': None, 'name': 'Сок свежевыжатый'}, ...",[Для приготовления коктейля необходимо три дол...
4,Гороховый суп с копченой грудинкой,Вкусный быстрый гороховый суп с копченой груди...,"[{'count': '2 л', 'name': 'Вода'}, {'count': '...",[]
...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,Очень простой рецепт домашней пиццы.,"[{'count': None, 'name': 'Мука пшеничная'}, {'...",[]
9996,Шоколадно-миндальный кекс с малиновым джемом,"Мои знакомые назвали этот кекс ""Негритянские г...","[{'count': '100 г', 'name': 'Миндаль'}, {'coun...","[Подготовить продукты., Миндаль залить кипятко..."
9997,Салат с редькой,Яркий хрустящий полезный салат с острым пикант...,"[{'count': None, 'name': 'Капуста белокочанная...",[Капусту нашинковать. Морковь и редьку потерет...
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",Ленивые вареники люблю с детства. Рецепт снова...,"[{'count': '440 г', 'name': 'Творог'}, {'count...",[Апельсин и лимон хорошенько промыла. Стерла ц...


In [55]:
for i in range(dataframe.shape[0]):
  steps = ' '.join(dataframe['cooking_steps'][i]).lower()
  dataframe.loc[i, 'cooking_steps'] = steps

In [56]:
dataframe['cooking_steps'][10]

'итак, в отличие от первоисточника (о нем читайте ниже), где предлагалось варить "сыр" из неотжатых овощей-фруктов, я готовлю из жмыха. я неоднократно пробовала следовать оригиналу, но результат меня не устроил, потому что получилось что-то вроде густого повидла. рассказываю, как поступаю я. отжимаю сок из моркови и слив. к слову, морковный и сливовый соки замечательно сочетаются, мой ребенок пьет такой коктейль с большим удовольствием. затем я протираю сливовую массу через сито (не люблю шкурку, даже измельченную) и смешиваю с морковной массой. в зависимости от вкуса моркови и слив я добавляю сахар и соль. вливаю 100 г воды, и на очень малом огне провариваю в кастрюли из нержавейки (удобно, не липнет). варю минут так 30. если ваша соковыжималка дает очень сухой жмых, есть смысл вместо воды долить часть сока. я добавляю воду, чтобы уменьшить сладость. если любите послаще, сливы вообще можно не отжимать, а измельчить и добавить в морковный жмых. и увеличить время варки. варю, пока масса

In [57]:
for i in range(dataframe.shape[0]):
  ing = []
  for l in range(len(dataframe['ingredients'][i])):
    ing.append(data['ingredients'][i][l]['name'].lower())
  dataframe.loc[i, 'ingredients'] = ing

In [58]:
dataframe

,title,description,ingredients,cooking_steps
0,Мини-кутабы,Весной и в начале лета практически все с больш...,"[мука пшеничная, вода, соль, зелень, сыр полут...","смешиваем просеянную муку и воду, солим. замеш..."
1,Куриные бедрышки с овощами и прованскими травами,Захотелось рецепт оформить роликом. Делала нес...,"[бедро куриное, картофель, помидоры черри, лук...",
2,Хлеб на яблочном соке,"Вы не пробовали хлебушек на соке? Мягкий, пуши...","[сок свежевыжатый, соль, мука пшеничная, вода,...",разводим дрожжи в теплой водичке и даем им под...
3,"Коктейль ""Застенчивая дыня""",Этот коктейль вовсе не для детей! Вкусно и хме...,"[сок свежевыжатый, вино белое полусладкое, мор...",для приготовления коктейля необходимо три доль...
4,Гороховый суп с копченой грудинкой,Вкусный быстрый гороховый суп с копченой груди...,"[вода, горох, грудинка, картофель, морковь, лу...",
...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,Очень простой рецепт домашней пиццы.,"[мука пшеничная, дрожжи, масло оливковое, соль...",
9996,Шоколадно-миндальный кекс с малиновым джемом,"Мои знакомые назвали этот кекс ""Негритянские г...","[миндаль, мука пшеничная, разрыхлитель теста, ...",подготовить продукты. миндаль залить кипятком ...
9997,Салат с редькой,Яркий хрустящий полезный салат с острым пикант...,"[капуста белокочанная, огурец, морковь, редька...",капусту нашинковать. морковь и редьку потереть...
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",Ленивые вареники люблю с детства. Рецепт снова...,"[творог, яйцо куриное, мука пшеничная, сахар, ...",апельсин и лимон хорошенько промыла. стерла це...


In [59]:
nltk.download('punkt_tab')
nltk.download('popular')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Down

True

In [60]:
stop_words = set(stopwords.words('russian'))

def remove_stopwords(text):
    tokens = word_tokenize(text, language='russian')
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words and word not in string.punctuation]
    return ' '.join(filtered_tokens)


dataframe['description'] = dataframe['description'].apply(remove_stopwords)
dataframe['cooking_steps'] = dataframe['cooking_steps'].apply(remove_stopwords)
dataframe # удаляем стоп-слова

,title,description,ingredients,cooking_steps
0,Мини-кутабы,весной начале лета практически большим удоволь...,"[мука пшеничная, вода, соль, зелень, сыр полут...",смешиваем просеянную муку воду солим замешивае...
1,Куриные бедрышки с овощами и прованскими травами,захотелось рецепт оформить роликом делала неск...,"[бедро куриное, картофель, помидоры черри, лук...",
2,Хлеб на яблочном соке,пробовали хлебушек соке мягкий пушистый аромат...,"[сок свежевыжатый, соль, мука пшеничная, вода,...",разводим дрожжи теплой водичке даем подняться ...
3,"Коктейль ""Застенчивая дыня""",коктейль вовсе детей вкусно хмельно ароматно у...,"[сок свежевыжатый, вино белое полусладкое, мор...",приготовления коктейля необходимо дольки дыни ...
4,Гороховый суп с копченой грудинкой,вкусный быстрый гороховый суп копченой грудинк...,"[вода, горох, грудинка, картофель, морковь, лу...",
...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,очень простой рецепт домашней пиццы,"[мука пшеничная, дрожжи, масло оливковое, соль...",
9996,Шоколадно-миндальный кекс с малиновым джемом,мои знакомые назвали кекс `` негритянские губк...,"[миндаль, мука пшеничная, разрыхлитель теста, ...",подготовить продукты миндаль залить кипятком 8...
9997,Салат с редькой,яркий хрустящий полезный салат острым пикантны...,"[капуста белокочанная, огурец, морковь, редька...",капусту нашинковать морковь редьку потереть кр...
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",ленивые вареники люблю детства рецепт снова вз...,"[творог, яйцо куриное, мука пшеничная, сахар, ...",апельсин лимон хорошенько промыла стерла цедру...


In [61]:
stemmer = SnowballStemmer("russian")

def stem(text):
  tokens = word_tokenize(text)
  lemmatized_words = [stemmer.stem(word) for word in tokens]
  return ' '.join(lemmatized_words)

dataframe['description'] = dataframe['description'].apply(stem)
dataframe['cooking_steps'] = dataframe['cooking_steps'].apply(stem)
dataframe # проводим лемматизацию

,title,description,ingredients,cooking_steps
0,Мини-кутабы,весн начал лет практическ больш удовольств ед ...,"[мука пшеничная, вода, соль, зелень, сыр полут...",смешива просея мук вод сол замешива однородн г...
1,Куриные бедрышки с овощами и прованскими травами,захотел рецепт оформ ролик дела нескольк месяц...,"[бедро куриное, картофель, помидоры черри, лук...",
2,Хлеб на яблочном соке,пробова хлебушек сок мягк пушист ароматн запах...,"[сок свежевыжатый, соль, мука пшеничная, вода,...",развод дрожж тепл водичк да подня шапочк реж я...
3,"Коктейль ""Застенчивая дыня""",коктейл вовс дет вкусн хмельн ароматн удачн со...,"[сок свежевыжатый, вино белое полусладкое, мор...",приготовлен коктейл необходим дольк дын попа о...
4,Гороховый суп с копченой грудинкой,вкусн быстр горохов суп копчен грудинк отличн ...,"[вода, горох, грудинка, картофель, морковь, лу...",
...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,очен прост рецепт домашн пицц,"[мука пшеничная, дрожжи, масло оливковое, соль...",
9996,Шоколадно-миндальный кекс с малиновым джемом,мо знаком назва кекс `` негритянск губк `` чес...,"[миндаль, мука пшеничная, разрыхлитель теста, ...",подготов продукт миндал зал кипятк 8-10 минут ...
9997,Салат с редькой,ярк хрустя полезн салат остр пикантн соус,"[капуста белокочанная, огурец, морковь, редька...",капуст нашинкова морков редьк потерет крупн те...
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",ленив вареник любл детств рецепт снов взял мо ...,"[творог, яйцо куриное, мука пшеничная, сахар, ...",апельсин лимон хорошеньк пром стерл цедр апель...


In [62]:
for i in range(dataframe.shape[0]):
  list_ing = dataframe['ingredients'][i]
  for t in range(len(list_ing)):
    list_ing[t] = remove_stopwords(list_ing[t] )
    list_ing[t]  = stem(list_ing[t] )
  dataframe['ingredients'][i] = list_ing

dataframe

Выходные данные были обрезаны до нескольких последних строк (5000).

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['ingredients'][i] = list_ing
<ipython-input-62-c7901c818882>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, t

,title,description,ingredients,cooking_steps
0,Мини-кутабы,весн начал лет практическ больш удовольств ед ...,"[мук пшеничн, вод, сол, зелен, сыр полутверд, ...",смешива просея мук вод сол замешива однородн г...
1,Куриные бедрышки с овощами и прованскими травами,захотел рецепт оформ ролик дела нескольк месяц...,"[бедр курин, картофел, помидор черр, лук красн...",
2,Хлеб на яблочном соке,пробова хлебушек сок мягк пушист ароматн запах...,"[сок свежевыжат, сол, мук пшеничн, вод, дрожж,...",развод дрожж тепл водичк да подня шапочк реж я...
3,"Коктейль ""Застенчивая дыня""",коктейл вовс дет вкусн хмельн ароматн удачн со...,"[сок свежевыжат, вин бел полусладк, морожен, к...",приготовлен коктейл необходим дольк дын попа о...
4,Гороховый суп с копченой грудинкой,вкусн быстр горохов суп копчен грудинк отличн ...,"[вод, горох, грудинк, картофел, морков, лук бе...",
...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,очен прост рецепт домашн пицц,"[мук пшеничн, дрожж, масл оливков, сол, вод, с...",
9996,Шоколадно-миндальный кекс с малиновым джемом,мо знаком назва кекс `` негритянск губк `` чес...,"[миндал, мук пшеничн, разрыхлител тест, коф ра...",подготов продукт миндал зал кипятк 8-10 минут ...
9997,Салат с редькой,ярк хрустя полезн салат остр пикантн соус,"[капуст белокоча, огурец, морков, редьк, горчи...",капуст нашинкова морков редьк потерет крупн те...
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",ленив вареник любл детств рецепт снов взял мо ...,"[творог, яйц курин, мук пшеничн, сахар, сол, а...",апельсин лимон хорошеньк пром стерл цедр апель...


In [63]:
import re

In [64]:
dataframe['cooking_steps_bio'] = np.nan # делаем метки bio

In [65]:
def bio_annotation(text, ingredients):
    # Разбиваем текст на токены (слова)
    tokens = re.findall(r'\w+|\W+', text)  # Учитываем слова и пробелы/знаки препинания
    annotated_tokens = []

    for token in tokens:
        if re.match(r'\s+|\W+', token) and not re.match(r'\w+', token):
              continue
        is_ingredient = False
        ingrid_1 = []
        ingrid_2 = []
        for i in ingredients:
          if len(i.split()) == 1:
            ingrid_1.append(i)
          else:
            i = i.split()
            ingrid_1.append(i[0])
            ingrid_2.append(i[1])

        for ing in ingredients:
            if token in ingrid_1:
                is_ingredient = True
                break
            if token in ingrid_2:
                is_ingredient = True
                break

        if is_ingredient:
            if token in ingrid_1:
                annotated_tokens.append((token, "B-ING"))
            else:
                annotated_tokens.append((token, "I-ING"))
        else:
            annotated_tokens.append((token, "O"))

    return annotated_tokens


for i in range(dataframe.shape[0]):
  text_cooking = dataframe['cooking_steps'][i]
  ingredients_cooking = dataframe['ingredients'][i]
  text_with_metka = bio_annotation(text_cooking, ingredients_cooking)
  dataframe['cooking_steps_bio'][i] = text_with_metka

dataframe['cooking_steps_bio']

<ipython-input-65-8ec1427afdcd>:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['cooking_steps_bio'][i] = text_with_metka
<ipython-input-65-8ec1427afdcd>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,cooking_steps_bio
0,"[(смешива, O), (просея, O), (мук, B-ING), (вод..."
1,[]
2,"[(развод, O), (дрожж, B-ING), (тепл, O), (води..."
3,"[(приготовлен, O), (коктейл, O), (необходим, O..."
4,[]
...,...
9995,[]
9996,"[(подготов, O), (продукт, O), (миндал, B-ING),..."
9997,"[(капуст, B-ING), (нашинкова, O), (морков, B-I..."
9998,"[(апельсин, B-ING), (лимон, B-ING), (хорошеньк..."


In [66]:
dataframe['description_bio'] = np.nan

In [67]:
dataframe

,title,description,ingredients,cooking_steps,cooking_steps_bio,description_bio
0,Мини-кутабы,весн начал лет практическ больш удовольств ед ...,"[мук пшеничн, вод, сол, зелен, сыр полутверд, ...",смешива просея мук вод сол замешива однородн г...,"[(смешива, O), (просея, O), (мук, B-ING), (вод...",NaN
1,Куриные бедрышки с овощами и прованскими травами,захотел рецепт оформ ролик дела нескольк месяц...,"[бедр курин, картофел, помидор черр, лук красн...",,[],NaN
2,Хлеб на яблочном соке,пробова хлебушек сок мягк пушист ароматн запах...,"[сок свежевыжат, сол, мук пшеничн, вод, дрожж,...",развод дрожж тепл водичк да подня шапочк реж я...,"[(развод, O), (дрожж, B-ING), (тепл, O), (води...",NaN
3,"Коктейль ""Застенчивая дыня""",коктейл вовс дет вкусн хмельн ароматн удачн со...,"[сок свежевыжат, вин бел полусладк, морожен, к...",приготовлен коктейл необходим дольк дын попа о...,"[(приготовлен, O), (коктейл, O), (необходим, O...",NaN
4,Гороховый суп с копченой грудинкой,вкусн быстр горохов суп копчен грудинк отличн ...,"[вод, горох, грудинк, картофел, морков, лук бе...",,[],NaN
...,...,...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,очен прост рецепт домашн пицц,"[мук пшеничн, дрожж, масл оливков, сол, вод, с...",,[],NaN
9996,Шоколадно-миндальный кекс с малиновым джемом,мо знаком назва кекс `` негритянск губк `` чес...,"[миндал, мук пшеничн, разрыхлител тест, коф ра...",подготов продукт миндал зал кипятк 8-10 минут ...,"[(подготов, O), (продукт, O), (миндал, B-ING),...",NaN
9997,Салат с редькой,ярк хрустя полезн салат остр пикантн соус,"[капуст белокоча, огурец, морков, редьк, горчи...",капуст нашинкова морков редьк потерет крупн те...,"[(капуст, B-ING), (нашинкова, O), (морков, B-I...",NaN
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",ленив вареник любл детств рецепт снов взял мо ...,"[творог, яйц курин, мук пшеничн, сахар, сол, а...",апельсин лимон хорошеньк пром стерл цедр апель...,"[(апельсин, B-ING), (лимон, B-ING), (хорошеньк...",NaN


In [68]:
for i in range(dataframe.shape[0]):
  text_desc = dataframe['description'][i]
  ingredients_desc = dataframe['ingredients'][i]
  text_with_metka = bio_annotation(text_desc, ingredients_desc)
  dataframe['description_bio'][i] = text_with_metka

<ipython-input-68-05ff737b119f>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['description_bio'][i] = text_with_metka
<ipython-input-68-05ff737b119f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [69]:
dataframe

,title,description,ingredients,cooking_steps,cooking_steps_bio,description_bio
0,Мини-кутабы,весн начал лет практическ больш удовольств ед ...,"[мук пшеничн, вод, сол, зелен, сыр полутверд, ...",смешива просея мук вод сол замешива однородн г...,"[(смешива, O), (просея, O), (мук, B-ING), (вод...","[(весн, O), (начал, O), (лет, O), (практическ,..."
1,Куриные бедрышки с овощами и прованскими травами,захотел рецепт оформ ролик дела нескольк месяц...,"[бедр курин, картофел, помидор черр, лук красн...",,[],"[(захотел, O), (рецепт, O), (оформ, O), (ролик..."
2,Хлеб на яблочном соке,пробова хлебушек сок мягк пушист ароматн запах...,"[сок свежевыжат, сол, мук пшеничн, вод, дрожж,...",развод дрожж тепл водичк да подня шапочк реж я...,"[(развод, O), (дрожж, B-ING), (тепл, O), (води...","[(пробова, O), (хлебушек, O), (сок, B-ING), (м..."
3,"Коктейль ""Застенчивая дыня""",коктейл вовс дет вкусн хмельн ароматн удачн со...,"[сок свежевыжат, вин бел полусладк, морожен, к...",приготовлен коктейл необходим дольк дын попа о...,"[(приготовлен, O), (коктейл, O), (необходим, O...","[(коктейл, O), (вовс, O), (дет, O), (вкусн, O)..."
4,Гороховый суп с копченой грудинкой,вкусн быстр горохов суп копчен грудинк отличн ...,"[вод, горох, грудинк, картофел, морков, лук бе...",,[],"[(вкусн, O), (быстр, O), (горохов, O), (суп, O..."
...,...,...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,очен прост рецепт домашн пицц,"[мук пшеничн, дрожж, масл оливков, сол, вод, с...",,[],"[(очен, O), (прост, O), (рецепт, O), (домашн, ..."
9996,Шоколадно-миндальный кекс с малиновым джемом,мо знаком назва кекс `` негритянск губк `` чес...,"[миндал, мук пшеничн, разрыхлител тест, коф ра...",подготов продукт миндал зал кипятк 8-10 минут ...,"[(подготов, O), (продукт, O), (миндал, B-ING),...","[(мо, O), (знаком, O), (назва, O), (кекс, O), ..."
9997,Салат с редькой,ярк хрустя полезн салат остр пикантн соус,"[капуст белокоча, огурец, морков, редьк, горчи...",капуст нашинкова морков редьк потерет крупн те...,"[(капуст, B-ING), (нашинкова, O), (морков, B-I...","[(ярк, O), (хрустя, O), (полезн, O), (салат, O..."
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",ленив вареник любл детств рецепт снов взял мо ...,"[творог, яйц курин, мук пшеничн, сахар, сол, а...",апельсин лимон хорошеньк пром стерл цедр апель...,"[(апельсин, B-ING), (лимон, B-ING), (хорошеньк...","[(ленив, O), (вареник, O), (любл, O), (детств,..."


In [70]:
def bio_to_entities(bio_data):
    entities = []
    current_entity = []
    for word, label in bio_data:
        if label == "B-ING":
            if current_entity:
                entities.append(" ".join(current_entity))
                current_entity = []
            current_entity.append(word)
        elif label == "I-ING":
            current_entity.append(word)
        elif label == "O" and current_entity:
            entities.append(" ".join(current_entity))
            current_entity = []
    if current_entity:
        entities.append(" ".join(current_entity))
    return entities

dataframe["true_entities"] = dataframe["description_bio"].apply(lambda x: bio_to_entities(x))
dataframe

,title,description,ingredients,cooking_steps,cooking_steps_bio,description_bio,true_entities
0,Мини-кутабы,весн начал лет практическ больш удовольств ед ...,"[мук пшеничн, вод, сол, зелен, сыр полутверд, ...",смешива просея мук вод сол замешива однородн г...,"[(смешива, O), (просея, O), (мук, B-ING), (вод...","[(весн, O), (начал, O), (лет, O), (практическ,...","[зелен, зелен, сыр, зелен, зелен]"
1,Куриные бедрышки с овощами и прованскими травами,захотел рецепт оформ ролик дела нескольк месяц...,"[бедр курин, картофел, помидор черр, лук красн...",,[],"[(захотел, O), (рецепт, O), (оформ, O), (ролик...",[]
2,Хлеб на яблочном соке,пробова хлебушек сок мягк пушист ароматн запах...,"[сок свежевыжат, сол, мук пшеничн, вод, дрожж,...",развод дрожж тепл водичк да подня шапочк реж я...,"[(развод, O), (дрожж, B-ING), (тепл, O), (води...","[(пробова, O), (хлебушек, O), (сок, B-ING), (м...",[сок]
3,"Коктейль ""Застенчивая дыня""",коктейл вовс дет вкусн хмельн ароматн удачн со...,"[сок свежевыжат, вин бел полусладк, морожен, к...",приготовлен коктейл необходим дольк дын попа о...,"[(приготовлен, O), (коктейл, O), (необходим, O...","[(коктейл, O), (вовс, O), (дет, O), (вкусн, O)...",[]
4,Гороховый суп с копченой грудинкой,вкусн быстр горохов суп копчен грудинк отличн ...,"[вод, горох, грудинк, картофел, морков, лук бе...",,[],"[(вкусн, O), (быстр, O), (горохов, O), (суп, O...","[грудинк, черн, перец, укроп, сол, грудинк, гр..."
...,...,...,...,...,...,...,...
9995,Пицца на тонком тесте с курицей и грибами,очен прост рецепт домашн пицц,"[мук пшеничн, дрожж, масл оливков, сол, вод, с...",,[],"[(очен, O), (прост, O), (рецепт, O), (домашн, ...",[]
9996,Шоколадно-миндальный кекс с малиновым джемом,мо знаком назва кекс `` негритянск губк `` чес...,"[миндал, мук пшеничн, разрыхлител тест, коф ра...",подготов продукт миндал зал кипятк 8-10 минут ...,"[(подготов, O), (продукт, O), (миндал, B-ING),...","[(мо, O), (знаком, O), (назва, O), (кекс, O), ...",[]
9997,Салат с редькой,ярк хрустя полезн салат остр пикантн соус,"[капуст белокоча, огурец, морков, редьк, горчи...",капуст нашинкова морков редьк потерет крупн те...,"[(капуст, B-ING), (нашинкова, O), (морков, B-I...","[(ярк, O), (хрустя, O), (полезн, O), (салат, O...",[соус]
9998,"Цитрусовые ленивые вареники ""Звездочки"" с соусом",ленив вареник любл детств рецепт снов взял мо ...,"[творог, яйц курин, мук пшеничн, сахар, сол, а...",апельсин лимон хорошеньк пром стерл цедр апель...,"[(апельсин, B-ING), (лимон, B-ING), (хорошеньк...","[(ленив, O), (вареник, O), (любл, O), (детств,...","[апельсин, лимон, творог]"


#Построение модели для извлечения именованных сущностей

In [71]:
from sklearn.model_selection import train_test_split

X_nottest, X_test = train_test_split(dataframe, test_size = 0.2, random_state=42)
X_train, X_val = train_test_split(X_nottest, test_size = 0.3, random_state=42)
X_train

,title,description,ingredients,cooking_steps,cooking_steps_bio,description_bio,true_entities
2983,Перец фаршированный жареной курочкой с гречкой...,банальн фарширова перц рис овощ надоел захотел...,"[перец болгарск, масл растительн, грудк курин,...",мелк пореза курин грудк обжар полуготовн расти...,"[(мелк, O), (пореза, O), (курин, I-ING), (груд...","[(банальн, O), (фарширова, O), (перц, O), (рис...",[черносл]
3488,Сладкий перец с чесноком,заготовк сладк перц красн цвет зим прост быстр...,"[перец сладк красн, чеснок, петрушк, масл раст...",перец вым очист семя разреза 4-6 част добав че...,"[(перец, B-ING), (вым, O), (очист, O), (семя, ...","[(заготовк, O), (сладк, I-ING), (перц, O), (кр...",[сладк]
5575,Горячий шоколад со снежками,люб горяч шоколад напиток-десерт обязательн по...,"[сахар коричнев, шоколад молочн, молок, маршме...",молок медлен нагрева кипен бер люб любим шокол...,"[(молок, B-ING), (медлен, O), (нагрева, O), (к...","[(люб, O), (горяч, O), (шоколад, B-ING), (напи...",[шоколад]
4937,"Торт ""Тёмная ночь""",предлага приготов очен вкусн торт `` темн ноч ...,"[яйц курин, сахар, кефир, сод, мук пшеничн, ка...",взбит яйц сахар добав кефир смеша сод добав ка...,"[(взбит, O), (яйц, B-ING), (сахар, B-ING), (до...","[(предлага, O), (приготов, O), (очен, O), (вку...",[смета]
9356,Милая сова из сахарной мастики,совушк №1 мастер-класс однажд случ совушк запо...,"[мастик, крахма]",торт нужн объедин мил совушек цветочк мстител ...,"[(торт, O), (нужн, O), (объедин, O), (мил, O),...","[(совушк, O), (1, O), (мастер, O), (класс, O),...",[]
...,...,...,...,...,...,...,...
8871,Котлеты из киноа со шпинатом и сыром,кино интересн круп не влюбля перв ложк дает бо...,"[кино, шпинат, яйц курин, сухар панировочн, му...",рецепт необходим круп кино отвар готовн шпинат...,"[(рецепт, O), (необходим, O), (круп, O), (кино...","[(кино, B-ING), (интересн, O), (круп, O), (не,...","[кино, кино]"
9826,Рис с курицей,fried rice and orange chicken рис люб дет зака...,"[рис, лук зелен, овощ, яйц курин, соев соус, ч...",,[],"[(fried, O), (rice, O), (and, O), (orange, O),...",[рис]
5268,"Салат ""Лисичкина шубка""",довольн нежн питательн салатик взгляд хорош ал...,"[морков, шампиньон, форел, яйц курин, картофел...",подготовлен продукт картофел яйц отвар морков ...,"[(подготовлен, O), (продукт, O), (картофел, B-...","[(довольн, O), (нежн, O), (питательн, O), (сал...",[]
9666,Мясная гармошка с ананасами и морковью,запечен мяс очен нежн вкус дела нов год основн...,"[свинин, морков, ананас, майонез, горчиц, прип...",мяс нареза гармошк натерет сол перц морков пор...,"[(мяс, O), (нареза, O), (гармошк, O), (натерет...","[(запечен, O), (мяс, O), (очен, O), (нежн, O),...",[]


In [72]:
def convert_to_spacy_format(data):
    spacy_data = []
    for text, bio_data in zip(data["description"], data["description_bio"]):
        entities = []
        start = 0
        for word, label in bio_data:
            end = start + len(word)
            if label == "B-ING":
                entities.append((start, end, "ING"))
            elif label == "I-ING":
                if entities and entities[-1][2] == "ING":
                    prev_start, prev_end, _ = entities[-1]
                    entities[-1] = (prev_start, end, "ING")
            start = end + 1
        spacy_data.append((text, {"entities": entities}))
    return spacy_data

train_data = convert_to_spacy_format(X_train)

In [73]:
!pip install spacy
!python -m spacy download ru_core_news_sm  # Для русского языка

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 42.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import tqdm
from tqdm import tqdm

In [74]:
import spacy
from spacy.training import Example
import random

nlp = spacy.load("ru_core_news_sm")
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")
ner.add_label("ING")


examples = []
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    examples.append(example)

optimizer = nlp.resume_training()
for itn in tqdm(range(10)):
    random.shuffle(examples)
    for example in examples:
        nlp.update([example], sgd=optimizer)

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "банальн фарширова перц рис овощ надоел захотел чег..." with entities "[(120, 127, 'ING')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "« уныл пор оч очарованье… » сам сер дождлив осен д..." with entities "[(97, 160, 'ING'), (161, 197, 'ING'), (198, 202, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "очен вкусн фа

In [75]:
def predict_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities

X_val["predicted_entities"] = X_val["description"].apply(predict_entities)

In [77]:
def calculate_iou(true_entities, pred_entities):
    true_set = set(true_entities)
    pred_set = set(pred_entities)
    intersection = true_set.intersection(pred_set)
    union = true_set.union(pred_set)
    return len(intersection) / len(union) if len(union) > 0 else 0

X_val["iou"] = X_val.apply(lambda row: calculate_iou(row["true_entities"], row["predicted_entities"]), axis=1)
mean_iou = X_val["iou"].mean()
print(f"Средний IoU description: {mean_iou:.2f}")

Средний IoU: 0.21


 # Оценка качества извлечения

In [83]:
X_test["predicted_entities"] = X_test["description"].apply(predict_entities)
X_test["iou"] = X_test.apply(lambda row: calculate_iou(row["true_entities"], row["predicted_entities"]), axis=1)
mean_iou = X_test["iou"].mean()
print(f"Средний IoU description: {mean_iou:.2f}") # как видим, метрика не отличается от валидации: 0.22. Что может говорить о том, что модель не переобучилась, но при этом видно, что недообучилась

Средний IoU description: 0.22
